In [13]:
from tensorflow.keras.layers import Dense, Flatten, MaxPool2D, Conv2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.activations import relu, softmax
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import csv
import os
import cv2

In [3]:
# Importing the csv into a dictionary
with open('alphabets_dataset/alphabet_labels.csv', mode='r', newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    gt = {rows[0]:rows[1] for rows in reader}


# Importing the dataset

# setting path and list variables
path = 'alphabets_dataset/alphabet_images/'
images = [] 
target = [] 

# loading dataset
for root, dirs, files in os.walk(path):
    for file in files:
        with open(os.path.join(root, file), "r") as auto:    
            im = cv2.imread(root+'\\'+file, 0)
            im = cv2.resize(im,(28,28))            
            images.append(im)
            target.append(gt[file])


images = np.array(images)
target = np.array(target)
images = images/255
import pandas as pd
targer = pd.DataFrame(target)
targer[0] = targer[0].map({'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9,'K':10,'L':11,'M':12,'N':13,'O':14,'P':15,'Q':16,'R':17,'S':18,'T':19,'U':20,'V':21,'W':22,'X':23,'Y':24,'Z':25})
y = targer[0].to_numpy()

In [4]:
#spliting into train and test
X_train, X_test, y_train, y_test = train_test_split(images,y, test_size=0.2, random_state=42)


In [5]:
# CNN model with three convolutional then pool layers and flatten then 2 dense layers

tf.random.set_seed(1234) # for consistent results
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'valid'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Flatten())
model.add(Dense(64,activation ="relu"))
model.add(Dense(128,activation ="relu"))


In [6]:
model.add(Dense(26,activation = 'softmax'))

In [7]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
history = model.fit(X_train, y_train, epochs=3)

In [9]:
model.save("Predict_letter.keras")

In [10]:
#loading and scoring model

model_load = tf.keras.models.load_model("Predict_letter.keras")

In [11]:
score= model_load.evaluate(X_test,y_test)

2328/2328 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9907 - loss: 0.0352
